In [1]:
a = 1
a
#https://medium.com/georgian-impact-blog/how-to-incorporate-tabular-data-with-huggingface-transformers-b70ac45fcfb4

1

In [2]:
# !pip install multimodal_transformers --user
# !pip uninstall -y transformers --user
# !pip install transformers==3.1.0 --user
# import transformers
# transformers.__version__

In [3]:
import datetime
import joblib

scale_col = "x_gaze_location_standard_scaled"

def apply_phq_cutoff(df , neg_phq_cutoff, pos_phq_cutoff):
    df["phq_binary_label"] = [0.0 if x <= neg_phq_cutoff else 1.0 if x >= pos_phq_cutoff else "other" for x in df.phq_score]
    df = df[df.phq_binary_label!= 'other']
    return df


def get_timecols_df_for_DL(fn ="/Users/orenkobo/Desktop/PhD_new/repos/HebLingStudy/notebooks/df.csv",
                           scale_col = scale_col,
                           word_embedding_dict_fn = "/Users/orenkobo/Desktop/PhD/HebLingStudy/Output_2021_Aug/word2alephbert_encodingdict.jbl"):

    import string, re
    print(f"{datetime.datetime.now()} Reading csv from {fn}")
    df = pd.read_csv(fn,
                     index_col=None,
                     converters={#'alephbert_enc': eval,
                         scale_col : eval,
                         # 'x_gaze_location_minmax_scaled' : eval,
                         # 'x_gaze_location_standard_scaled' : eval,
                         # 'target_word_x_range' : eval
                         # 'phq_label': bool
                     })
    print(df.shape)

    def exclude_char(s):
        exclude = set(string.punctuation)
        exclude.remove("-")
        return ''.join(ch for ch in s if ch not in exclude)

    words_embedding_dict = joblib.load("word2alephbert_encodingdict.jbl")
    df = df[df.Sentence_type != 'F'].reset_index(drop=True)
    df["words_order"] = df["words_order"].apply(lambda x : [exclude_char(w) for w in eval(x)])
    df['word_embeddings_order'] = df["words_order"].apply(lambda x : [words_embedding_dict[w] for w in x])

    id_cols = ["phq_score","phq_group","Subject", "Sentence_type",
               "sentence_pupil_diameter_mean","set_num", "words_order","word_embeddings_order"]
    # vec_size = 3500
    # new_colname = f"x_gaze_location_{vec_size}"
    cols = [f"timepoint#{i}" for i in range(875)]
    # df[new_colname] = df["x_gaze_location_standard_scaled"].apply(lambda x : x[:vec_size])
    timeseries_df = pd.DataFrame(data = df[scale_col].to_list() , columns = cols)
    timeseries_df[id_cols] = df[id_cols]
    timeseries_df = timeseries_df.iloc[:,200:]
    cols = [x for x in timeseries_df.columns if "timepoint" in x]
    return timeseries_df, cols

In [4]:
# from OutputsAnalyser.run_timepoints_utils import get_timecols_df_for_DL, apply_phq_cutoff
import pandas as pd
d = {}
df_fn = "/export/home/orenkobo/Aim1/paper_analysis/Artifacts/df_new_full__unsegmented_alldata_new_FINAL.csv"
# df_fn = "/Users/orenkobo/Desktop/PhD/HebLingStudy/ts_data/Artifacts2/df_new_full__unsegmented_alldata_new_FINAL.csv"
et_scale_col = "x_gaze_location"
override_cutoff = [7,8]
df, timepoint_cols = get_timecols_df_for_DL(fn =df_fn, scale_col = et_scale_col)
df = apply_phq_cutoff(df,
                      neg_phq_cutoff = override_cutoff[0],
                      pos_phq_cutoff = override_cutoff[1])


cond_df = pd.get_dummies(df['Sentence_type'])
cond_cols = cond_df.columns.tolist()
df = pd.concat([df, cond_df],axis=1)
df = df.reset_index(drop=True)
df

2022-03-27 14:30:23.024512 Reading csv from /export/home/orenkobo/Aim1/paper_analysis/Artifacts/df_new_full__unsegmented_alldata_new_FINAL.csv
(9696, 31)


FileNotFoundError: [Errno 2] No such file or directory: 'word2alephbert_encodingdict.jbl'

In [ ]:
df.shape

In [ ]:
import numpy as np
from dataclasses import dataclass, field
import json
import logging
import os
from typing import Optional

import numpy as np
import pandas as pd
from transformers import (
    AutoTokenizer,
    AutoConfig,
    Trainer,
    EvalPrediction,
    set_seed
)
from transformers.training_args import TrainingArguments
from transformers import BertTokenizer

from multimodal_transformers.data import load_data_from_folder
from multimodal_transformers.model import AutoModelWithTabular

from multimodal_transformers.model import TabularConfig

logging.basicConfig(level=logging.INFO)
os.environ['COMET_MODE'] = 'DISABLED'


In [ ]:
data_df = df.copy()

In [ ]:
train_df, val_df, test_df = np.split(data_df.sample(frac=1), [int(.8*len(data_df)), int(.9 * len(data_df))])
print('Num examples train-val-test')
print(len(train_df), len(val_df), len(test_df))
train_df.to_csv('train.csv')
val_df.to_csv('val.csv')
test_df.to_csv('test.csv')

In [ ]:
@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
    """

    model_name_or_path: str = field(
        metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    cache_dir: Optional[str] = field(
        default=None, metadata={"help": "Where do you want to store the pretrained models downloaded from s3"}
    )


@dataclass
class MultimodalDataTrainingArguments:
    """
    Arguments pertaining to how we combine tabular features
    Using `HfArgumentParser` we can turn this class
    into argparse arguments to be able to specify them on
    the command line.
    """

    data_path: str = field(metadata={
        'help': 'the path to the csv file containing the dataset'
    })
    column_info_path: str = field(
        default=None,
        metadata={
            'help': 'the path to the json file detailing which columns are text, categorical, numerical, and the label'
        })

    column_info: dict = field(
        default=None,
        metadata={
            'help': 'a dict referencing the text, categorical, numerical, and label columns'
                    'its keys are text_cols, num_cols, cat_cols, and label_col'
        })

    categorical_encode_type: str = field(default='ohe',
                                         metadata={
                                             'help': 'sklearn encoder to use for categorical data',
                                             'choices': ['ohe', 'binary', 'label', 'none']
                                         })
    numerical_transformer_method: str = field(default='yeo_johnson',
                                              metadata={
                                                  'help': 'sklearn numerical transformer to preprocess numerical data',
                                                  'choices': ['yeo_johnson', 'box_cox', 'quantile_normal', 'none']
                                              })
    task: str = field(default="classification",
                      metadata={
                          "help": "The downstream training task",
                          "choices": ["classification", "regression"]
                      })

    mlp_division: int = field(default=4,
                              metadata={
                                  'help': 'the ratio of the number of '
                                          'hidden dims in a current layer to the next MLP layer'
                              })
    combine_feat_method: str = field(default='individual_mlps_on_cat_and_numerical_feats_then_concat',
                                     metadata={
                                         'help': 'method to combine categorical and numerical features, '
                                                 'see README for all the method'
                                     })
    mlp_dropout: float = field(default=0.1,
                               metadata={
                                   'help': 'dropout ratio used for MLP layers'
                               })
    numerical_bn: bool = field(default=True,
                               metadata={
                                   'help': 'whether to use batchnorm on numerical features'
                               })
    use_simple_classifier: str = field(default=True,
                                       metadata={
                                           'help': 'whether to use single layer or MLP as final classifier'
                                       })
    mlp_act: str = field(default='relu',
                         metadata={
                             'help': 'the activation function to use for finetuning layers',
                             'choices': ['relu', 'prelu', 'sigmoid', 'tanh', 'linear']
                         })
    gating_beta: float = field(default=0.2,
                               metadata={
                                   'help': "the beta hyperparameters used for gating tabular data "
                                           "see https://www.aclweb.org/anthology/2020.acl-main.214.pdf"
                               })

    def __post_init__(self):
        assert self.column_info != self.column_info_path
        if self.column_info is None and self.column_info_path:
            with open(self.column_info_path, 'r') as f:
                self.column_info = json.load(f)

In [ ]:
cat_feats = ['A','B','C','D']
num_feats = ['sentence_pupil_diameter_mean']
nlp_feats = ['words_order']

text_cols = nlp_feats# ['Title', 'Review Text']
cat_cols = cat_feats# ['Clothing ID', 'Division Name', 'Department Name', 'Class Name']
numerical_cols = num_feats#['Rating', 'Age', 'Positive Feedback Count']

column_info_dict = {
    'text_cols': text_cols,
    'num_cols': numerical_cols,
    'cat_cols': cat_cols,
    'label_col': 'phq_binary_label',
    'label_list': [0,1]
}


model_args = ModelArguments(
    # model_name_or_path= '/Users/orenkobo/Desktop/PhD/Aim1/LM/alephbert-base' #'bert-base-uncased'
    model_name_or_path= "/export/home/orenkobo/Aim1/paper_analysis/LM/alephbert-base"
)

data_args = MultimodalDataTrainingArguments(
    data_path='.',
    combine_feat_method='gating_on_cat_and_num_feats_then_sum',
    column_info=column_info_dict,
    task='classification'
)

training_args = TrainingArguments(
    output_dir="./multimodal_logs/model_name",
    logging_dir="./multimodal_logs/runs",
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=32,
    num_train_epochs=5,
    evaluate_during_training=True,
    logging_steps=25,
    eval_steps=250
)

set_seed(training_args.seed)

In [ ]:

tokenizer = BertTokenizer.from_pretrained("/export/home/orenkobo/Aim1/paper_analysis/LM/alephbert-base")


In [ ]:
# Get Datasets
train_dataset, val_dataset, test_dataset = load_data_from_folder(
    data_args.data_path,
    data_args.column_info['text_cols'],
    tokenizer,
    label_col=data_args.column_info['label_col'],
    label_list=data_args.column_info['label_list'],
    categorical_cols=data_args.column_info['cat_cols'],
    numerical_cols=data_args.column_info['num_cols'],
    sep_text_token_str=tokenizer.sep_token,
)

In [ ]:
num_labels = len(np.unique(train_dataset.labels))


config = AutoConfig.from_pretrained(
    model_args.config_name if model_args.config_name else model_args.model_name_or_path,
    cache_dir=model_args.cache_dir,
)
tabular_config = TabularConfig(num_labels=num_labels,
                               cat_feat_dim=train_dataset.cat_feats.shape[1],
                               numerical_feat_dim=train_dataset.numerical_feats.shape[1],
                               **vars(data_args))
config.tabular_config = tabular_config

In [ ]:
model = AutoModelWithTabular.from_pretrained(
    model_args.config_name if model_args.config_name else model_args.model_name_or_path,
    config=config,
    cache_dir=model_args.cache_dir
)
model

In [ ]:
import numpy as np
from scipy.special import softmax
from sklearn.metrics import auc,  precision_recall_curve, roc_auc_score, accuracy_score, f1_score, confusion_matrix, matthews_corrcoef

def calc_classification_metrics(p: EvalPrediction):
    pred_labels = np.argmax(p.predictions, axis=1)
    pred_scores = softmax(p.predictions, axis=1)[:, 1]
    labels = p.label_ids

    roc_auc_pred_score = roc_auc_score(labels, pred_scores)
    acc_score = accuracy_score(labels, pred_labels)
    precisions, recalls, thresholds = precision_recall_curve(labels,
                                                             pred_scores)
    fscore = (2 * precisions * recalls) / (precisions + recalls)
    fscore[np.isnan(fscore)] = 0
    ix = np.argmax(fscore)
    threshold = thresholds[ix].item()
    pr_auc = auc(recalls, precisions)
    tn, fp, fn, tp = confusion_matrix(labels, pred_labels, labels=[0, 1]).ravel()
    result = {'roc_auc': roc_auc_pred_score,
              'threshold': threshold,
              'pr_auc': pr_auc,
              'recall': recalls[ix].item(),
              'accuracy' : acc_score,
              'precision': precisions[ix].item(), 'f1': fscore[ix].item(),
              'tn': tn.item(), 'tp': tp.item(), 'fp': fp.item(), 'fn': fn.item()
              }


    return result

In [ ]:
a = 2
b = 3
a

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=calc_classification_metrics,
)
trainer

In [ ]:
import datetime
print(datetime.datetime.now())

In [ ]:
%%time
trainer.train()

In [ ]:
print(datetime.datetime.now())

In [ ]:
from pprint import pformat
i = 0
if training_args.do_eval:
    print("*** Evaluate ***")
    eval_result = trainer.evaluate(eval_dataset=val_dataset)
    print(pformat(eval_result, indent=4))
    output_eval_file = os.path.join(
        training_args.output_dir, f"eval_metric_results_fold_{i+1}.txt"
    )

    # with open(output_eval_file, "w") as writer:
    #     print("***** Eval results *****")
    #     for key, value in eval_result.items():
    #         print("  %s = %s", key, value)
    #         writer.write("%s = %s\n" % (key, value))

In [ ]:
predictions_probs = trainer.predict(test_dataset=test_dataset).predictions
predictions = np.argmax(predictions_probs, axis=1)
predictions

In [ ]:
accuracy = (true_values == predictions).sum() / N


In [ ]:
def train_fold():
    pass

In [26]:
import torch

with torch.no_grad():
    _, logits, classifier_outputs = model(
        model_inputs['input_ids'],
        token_type_ids=model_inputs['token_type_ids'],
        cat_feats=categorical_feat,
        numerical_feats=numerical_feat
    )

NameError: name 'model_inputs' is not defined

In [ ]:
a = 2
a

In [ ]:
# from transformers import BertConfig
# import torch
# from multimodal_transformers.model import BertWithTabular
# from multimodal_transformers.model import TabularConfig
# from transformers import BertTokenizer
# from sklearn.model_selection import LeavePGroupsOut
#
# tokenizer = BertTokenizer.from_pretrained("/Users/orenkobo/Desktop/PhD/Aim1/LM/alephbert-base")
# bert_config = BertConfig.from_pretrained('/Users/orenkobo/Desktop/PhD/Aim1/LM/alephbert-base')
#

# cat_feats = ['A','B','C','D']
# num_feats = ['sentence_pupil_diameter_mean']
# nlp_feat = 'words_order'
#
# tabular_config = TabularConfig(
#     combine_feat_method='attention_on_cat_and_numerical_feats',  # https://multimodal-toolkit.readthedocs.io/en/latest/notes/combine_methods.html combining tabular data
#     cat_feat_dim=len(cat_feats),  # need to specify this
#     numerical_feat_dim=len(num_feats),  # need to specify this
#     num_labels=2,   # need to specify this, assuming our task is binary classification
#     use_num_bn=False,
# )
# 'onlplab/alephbert-base'


# X = df[cat_feats + num_feats + [nlp_feat]]
# y = df['phq_binary_label']
# groups = df['Subject']


# lpgo = LeavePGroupsOut(n_groups=10)
# lpgo.get_n_splits(groups=groups)

# i = 0
# for train_index, test_index in lpgo.split(X = X , y = y, groups = groups):
#     if i == 1:
#         break
#
#     X_train, X_test = X.iloc[train_index], X.iloc[test_index]
#     y_train, y_test = y.iloc[train_index], y.iloc[test_index]
#
#     train_texts = [" ".join(x) for x in X_train[nlp_feat]]
#     train_model_inputs = tokenizer(train_texts, padding=True, padding_side='left')
#     print("AAAAAAA1")
#     train_numerical_feat = torch.tensor(X_train[num_feats].values).float()
#     train_categorical_feat = torch.tensor(X_train[cat_feats].values).float()
#     print("AAAAAAA2")
#     train_model_inputs['cat_feats'] = train_categorical_feat
#     train_model_inputs['num_feats'] = train_numerical_feat
#     labels = torch.tensor(y_train.values)
#     train_model_inputs['labels'] = labels
#     print("AAAAAAA3")
#     test_texts = [" ".join(x) for x in X_test[nlp_feat]]
#     test_model_inputs = tokenizer(test_texts, padding=True)
#     test_numerical_feat = torch.tensor(X_test[num_feats].values).float()
#     test_categorical_feat = torch.tensor(X_test[cat_feats].values).float()
#
#     test_model_inputs['cat_feats'] = test_categorical_feat
#     test_model_inputs['num_feats'] = test_numerical_feat
#     print("AAAAAAA4")
#     bert_config.tabular_config = tabular_config
#
#     model = BertWithTabular.from_pretrained('/Users/orenkobo/Desktop/PhD/Aim1/LM/alephbert-base', config=bert_config)
#     print("AAAAAAA5")
#     i+=1

# loss, logits, layer_outs = model(
#     torch.tensor(train_model_inputs['input_ids']),
#     token_type_ids=torch.tensor(train_model_inputs['token_type_ids']),
#     labels=labels,
#     cat_feats=train_categorical_feat,
#     numerical_feats=train_numerical_feat
# )
# print("AAAAAAA6")
# print("AAAAAAA6")
# model

#with torch.no_grad():
#    _, logits, classifier_outputs = model(
#        torch.tensor(test_model_inputs['input_ids']),
#        token_type_ids=torch.tensor(test_model_inputs['token_type_ids']),
#        cat_feats=test_categorical_feat,
#        numerical_feats=test_numerical_feat

#    )#%%